# A demonstration on defining the vocabulary.

## Preliminaries

Setting the classification tolerance in ppm; dismissing entities below the intensity floor in <?>.

In [2]:
import json
import pymzml
import numpy as np

INTENSITY_FLOOR = 10
TOLERANCE_FACTOR = 7
# DATA_PATH = "../data/"
DATA_PATH = '/Users/simon/Dropbox/MS_Ink_Data/Alphabet/'
DATA_FILENAME = "abcdefgh_1.mzML"

An auxiliary method for classification within a certain range.

In [3]:
def classify(classes, feature):
    for class_ in classes:
        if feature >= class_[0] and feature <= class_[1]:
            return str(class_)

Downloading the data.

In [ ]:
import os
import urllib.request

URL = "https://www.dropbox.com/s/h89znvb6okbfebb/abcdefgh_1.mzML?dl=1"

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

urllib.request.urlretrieve(URL, DATA_PATH + DATA_FILENAME)

## Initialisation

Running the data.

In [4]:
run = pymzml.run.Reader(DATA_PATH + DATA_FILENAME)

Reading entities in the form of (mass, intensity) tuples.

In [5]:
preprocessed_mnis = []  
for spectrum in run:
    for mass, intensity in spectrum.peaks:
        if intensity > INTENSITY_FLOOR:
            preprocessed_mnis.append((mass, intensity, spectrum['id']))

Using numpy to sort the tuples in ascending mass order.

In [6]:
mnis_dtype = [('mass', float), ('intensity', float), ('id', int)]
mnis = np.array(preprocessed_mnis, dtype=mnis_dtype)
mnis.sort(order='mass') 

## Classification

Extracting the words.

In [7]:
words = []
starting_class_mass = mnis[0][0]
previous_mass = mnis[0][0]
for entity in mnis:
    mass = entity[0]
    tolerance = (previous_mass / 1000000) * TOLERANCE_FACTOR
    if mass - previous_mass > tolerance:
        words.append((starting_class_mass, previous_mass))
        starting_class_mass = mass
    previous_mass = mass 
words.append((starting_class_mass, previous_mass))

print(len(words))

2442


Building the vocabulary.

In [8]:
vocabulary = {}
for entity in mnis:
    key = str(entity[2])
    if key not in vocabulary:
        vocabulary[key] = {}
    class_ = classify(words, entity[0])
    if class_ not in vocabulary[key]:
        vocabulary[key][class_] = 0
    vocabulary[key][class_] += entity[1]
    
print(len(vocabulary))

KeyboardInterrupt: 

Create and run the LDA object

In [9]:
import sys
sys.path.append('/Users/simon/git/lda/code')

In [10]:
from lda import VariationalLDA

In [11]:
v_lda = VariationalLDA(corpus = vocabulary,K=10) # corpus would be a better name than vocabulary

Found 1498 unique words
Object created with 6327 documents


In [12]:
# Run for 100 iterations
v_lda.run_vb(n_its = 10)

Initialising
Starting iterations
Iteration 0 (change = 13.4390557542) (8.505446 seconds, I think I'll finish in 1.41757433333 minutes)
Iteration 1 (change = 0.14299543463) (8.548027 seconds, I think I'll finish in 1.28220405 minutes)
Iteration 2 (change = 0.271475978366) (8.216885 seconds, I think I'll finish in 1.09558466667 minutes)
Iteration 3 (change = 0.403321748745) (8.513158 seconds, I think I'll finish in 0.993201766667 minutes)
Iteration 4 (change = 0.541589556047) (8.596392 seconds, I think I'll finish in 0.8596392 minutes)
Iteration 5 (change = 0.680106419022) (8.344969 seconds, I think I'll finish in 0.695414083333 minutes)
Iteration 6 (change = 0.792087038232) (8.187309 seconds, I think I'll finish in 0.5458206 minutes)
Iteration 7 (change = 0.836844300185) (8.288156 seconds, I think I'll finish in 0.4144078 minutes)
Iteration 8 (change = 0.787552194539) (8.799662 seconds, I think I'll finish in 0.293322066667 minutes)
Iteration 9 (change = 0.668316740552) (8.601123 second

In [13]:
a = v_lda.get_topic_as_dict(0)

In [14]:
print a

{'(151.09847872359833, 151.14774846579857)': 2.4039698078075154e-05, '(277.98797509867501, 278.06580782869469)': 4.8151630995122737e-05, '(272.12429753288126, 272.16606638851204)': 1.1087108023884853e-05, '(198.06762349229547, 198.13858880026723)': 4.4127372333565678e-06, '(381.73894958401314, 382.77873954256165)': 0.00034187519498437994, '(263.75916279237083, 263.77428016266151)': 3.4194673985305084e-07, '(441.03770890187093, 441.74079945490018)': 0.0013855175650435317, '(310.06312272346935, 310.59277614054275)': 0.0019224792093145203, '(151.65155952817719, 151.66494106806476)': 9.4778587772221736e-06, '(329.39202320241634, 329.43997564535005)': 4.9388611186653631e-05, '(216.47675289226663, 216.52580813911075)': 1.6073364208863335e-05, '(155.14480906525327, 155.25315106052352)': 4.2949590691691271e-05, '(311.50442007932457, 311.59909906775709)': 5.4180417653639422e-05, '(216.83448881376145, 216.91509305479178)': 5.2297664723624435e-05, '(399.52964759546626, 399.97324079978398)': 8.510